In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from zipfile import ZipFile
import pandas as pd
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import average_precision_score, roc_auc_score, precision_recall_fscore_support
from sklearn.metrics import multilabel_confusion_matrix,classification_report

In [0]:
# Change the path of the data file
zip_ref = ZipFile("/content/gdrive/My Drive/Co_lab/Blog/blog-authorship-corpus.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [4]:
!ls -l /tmp

total 781668
-rw-r--r-- 1 root root 800419647 Apr  1 15:46 blogtext.csv
srw------- 1 root root         0 Apr  1 15:45 drivefs_ipc.0
srw------- 1 root root         0 Apr  1 15:45 drivefs_ipc.0_shell
drwx------ 2 root root      4096 Apr  1 15:45 tmp80dzot4f


In [0]:
df = pd.read_csv('/tmp/blogtext.csv')

In [6]:
df.shape

(681284, 7)

In [7]:
df.head(5)

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [0]:
from sklearn.utils import resample
blog = resample(df,replace=True,n_samples=50000,random_state=7) # with replacement

In [9]:
blog.shape

(50000, 7)

In [10]:
blog.head(5)

,id,gender,age,topic,sign,date,text
585903,944569,female,14,Student,Leo,"02,February,2003","fine, Nini (hehe omg, i called u th..."
189636,1862467,female,27,indUnk,Cancer,"17,July,2004",How to tell Nepal's Maoist rebels from...
573977,1386800,female,45,Non-Profit,Pisces,"29,June,2004",Quotes on Friendship 'Each frie...
535030,3093428,female,17,Student,Taurus,"03,June,2004",Ben!!! Welcome to da club! No...
328599,240089,male,24,indUnk,Taurus,"04,May,2003",Happy Drinko De Mayo... Let us cele...


In [0]:
#Preprocessing

In [0]:
import re

In [0]:
#converting text into lowercase

In [14]:
blog['text'] = [text.lower() for text in blog['text']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
blog['text'][0:5]

585903               fine, nini (hehe omg, i called u th...
189636            how to tell nepal's maoist rebels from...
573977                quotes on friendship    'each frie...
535030                     ben!!! welcome to da club! no...
328599               happy drinko de mayo... let us cele...
Name: text, dtype: object

In [0]:
#Removing unwanted charecters

In [17]:
blog['text'] = [re.sub("[^a-z0-9 ]+","", text) for text in blog['text']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
blog['text'][0:5]

585903               fine nini hehe omg i called u that ...
189636            how to tell nepals maoist rebels from ...
573977                quotes on friendship    each frien...
535030                     ben welcome to da club now to...
328599               happy drinko de mayo let us celebra...
Name: text, dtype: object

In [0]:
#Removing Unwanted spaces

In [20]:
blog['text'] = [re.sub("\s+"," ", text).strip() for text in blog['text']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
blog['text'][0:5]

585903    fine nini hehe omg i called u that i wont put ...
189636    how to tell nepals maoist rebels from gardenva...
573977    quotes on friendship each friend represents a ...
535030    ben welcome to da club now to just get others ...
328599    happy drinko de mayo let us celebrate the inde...
Name: text, dtype: object

In [22]:
import nltk
from nltk import word_tokenize, pos_tag
nltk.download('wordnet')
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
from nltk import punkt

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
#removing stop words

In [24]:
stop = stopwords.words('english')
blog['text'] = blog['text'].apply( lambda t : " ".join( word for word in t.split() 
                                                       if len(word)>2 if word not in stop ) )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [25]:
blog['text'][0:5]

585903    fine nini hehe omg called wont put test scores...
189636    tell nepals maoist rebels gardenvariety commun...
573977    quotes friendship friend represents world worl...
535030    ben welcome club get others insomeones gotta h...
328599    happy drinko mayo let celebrate independence 6...
Name: text, dtype: object

In [26]:
#### Lemmatize the text

#Lemmatization
lemmatizer = WordNetLemmatizer()
# perform lemmatization on the title
blog['text'] = blog['text'].apply( lambda t : " ".join( [lemmatizer.lemmatize(word) for word in t.split()  ]) )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
blog['text'][0:6]

585903    fine nini hehe omg called wont put test score ...
189636    tell nepal maoist rebel gardenvariety communis...
573977    quote friendship friend represents world world...
535030    ben welcome club get others insomeones gotta h...
328599    happy drinko mayo let celebrate independence 6...
79463     lady ilene lived distilled kerosene started ab...
Name: text, dtype: object

In [28]:
#As we want to make this into a multi-label classification problem, you are required to merge all the label columns together, so that we have all the labels together for a particular sentence
blog.columns

Index(['id', 'gender', 'age', 'topic', 'sign', 'date', 'text'], dtype='object')

In [29]:
blog.head(2)

,id,gender,age,topic,sign,date,text
585903,944569,female,14,Student,Leo,"02,February,2003",fine nini hehe omg called wont put test score ...
189636,1862467,female,27,indUnk,Cancer,"17,July,2004",tell nepal maoist rebel gardenvariety communis...


In [30]:
#### a. Label columns to merge: “gender”, “age”, “topic”, “sign”

#Merge all four labels into one label

blog['labels'] = blog.apply(lambda x : [x['gender'], str(x['age']), x['topic'], x['sign']], axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [31]:
blog[['text','labels']][0:2]

,text,labels
585903,fine nini hehe omg called wont put test score ...,"[female, 14, Student, Leo]"
189636,tell nepal maoist rebel gardenvariety communis...,"[female, 27, indUnk, Cancer]"


In [0]:
#### b. After completing the previous step, there should be only two columns in your dataframe i.e. “text” and “labels”

#Create a DataFrame with two columns text and length

blog_df = pd.DataFrame()
blog_df['text'] = blog['text']
blog_df['labels'] = blog['labels']

In [34]:
blog_df.shape

blog_df.columns

blog_df.head()

,text,labels
585903,fine nini hehe omg called wont put test score ...,"[female, 14, Student, Leo]"
189636,tell nepal maoist rebel gardenvariety communis...,"[female, 27, indUnk, Cancer]"
573977,quote friendship friend represents world world...,"[female, 45, Non-Profit, Pisces]"
535030,ben welcome club get others insomeones gotta h...,"[female, 17, Student, Taurus]"
328599,happy drinko mayo let celebrate independence 6...,"[male, 24, indUnk, Taurus]"


In [0]:
### 4. Separate features and labels, and split the data into training and testing (5 points)

#Features
X = blog_df['text']

In [0]:
#labels
y = blog_df['labels']

In [0]:
#Split Features and labels into training set and test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=7)


In [44]:
#Shape of X_train and y_train

X_train.shape, y_train.shape


((40000,), (40000,))

In [45]:
#Shape of X_test and y_test

X_test.shape, y_test.shape

((10000,), (10000,))

In [0]:
### 5. Vectorize the features (5 points)

#### a. Create a Bag of Words using count vectorizer
#### i. Use ngram_range=(1, 2)
#### ii. Vectorize training and testing features

#Create an instance of CountVectorizer

countvect = CountVectorizer(ngram_range=(1,2), max_features=8000)

In [47]:
#Fit the CountVectorizer on Feature set

countvect.fit(X)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=8000, min_df=1,
                ngram_range=(1, 2), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [0]:
#Transform training set and test set into count vectors (BoW)

X_bow_train = countvect.transform(X_train)
X_bow_test = countvect.transform(X_test)

In [49]:
#Shape of sparse csr matrices with count vectors of training set and test set

X_bow_train.shape, X_bow_test.shape

((40000, 8000), (10000, 8000))

In [0]:
#### b. Print the term-document matrix

#Function to create term-document matrix by chunking the sparse csr matrix

def get_dtm_using_chunks(sparce_csr_matrix, size, chunk_size):
    #size=countvect.max_features
    #chunk_size=100
    dtm = pd.DataFrame()
    if size < chunk_size:
        dtm = pd.concat([dtm, pd.DataFrame(sparce_csr_matrix[:,0:size].toarray())], axis=1)
        gc.collect()
    chunks_nb = int(size/chunk_size)
    print("Max. Number of Features : ", size)
    print("Number of chunks used : ", chunks_nb)
    iter_ints = range(0, chunks_nb)
    print("Started creating DTM matrix by chunking....")
    for i in iter_ints:
        j = i * chunk_size
        if i+1 < chunks_nb:
            k = j + chunk_size
            #print("k loop",i,j,k,"\t")
            dtm = pd.concat([dtm, pd.DataFrame(sparce_csr_matrix[:,j:k].toarray())], axis=1)
            gc.collect()
        else:
            #print("j loop",i,j,"\t")
            dtm = pd.concat([dtm, pd.DataFrame(sparce_csr_matrix[:,j:].toarray())], axis=1)
            gc.collect()
    print("...Done")
    return dtm

In [0]:
import gc

In [54]:
print("Term-Document Matrix for Training-set count vector:")
dtm_train = get_dtm_using_chunks(X_bow_train, countvect.max_features, 100)

Term-Document Matrix for Training-set count vector:
Max. Number of Features :  8000
Number of chunks used :  80
Started creating DTM matrix by chunking....
...Done


In [55]:
print("Term-Document Matrix for Test-set count vector:")
dtm_test = get_dtm_using_chunks(X_bow_test, countvect.max_features, 100)

Term-Document Matrix for Test-set count vector:
Max. Number of Features :  8000
Number of chunks used :  80
Started creating DTM matrix by chunking....
...Done


In [56]:
##### Term-Document Matrix for training set

dtm_train.columns = countvect.get_feature_names()
dtm_train.shape

(40000, 8000)

In [57]:
dtm_train.columns

dtm_train.head()

##### Term-Document Matrix for test set

dtm_test.columns = countvect.get_feature_names()
dtm_test.shape

(10000, 8000)

In [58]:
dtm_test.columns

dtm_test.head()

,100,1000,101,1030,10th,1100,1130,11th,120,1230,130,13th,14th,150,151,15th,17th,18th,1996,1998,1999,1st,200,2000,2001,2002,2003,2004,2005,20th,21st,230,2day,2nd,300,3000,316,316 finisher,330,3am,...,yoga,york,york city,york time,youd,youd like,youd think,youll,youll find,youll never,youll see,younbsp,young,young man,young people,young woman,younger,youngest,youre,youre going,youre gonna,youre one,youth,youth group,youve,youve ever,youve got,yr,yuck,yum,yummy,yun,yung,yup,zach,zealand,zero,zombie,zone,zoo
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [59]:
### 6. Create a dictionary to get the count of every label i.e. the key will be label name and value will be the total count of the label. (5 points)

y_train.head()

y_test.head()

276269             [female, 25, Arts, Virgo]
471190      [male, 14, Student, Sagittarius]
526347    [female, 17, Consulting, Aquarius]
129326            [male, 24, indUnk, Pisces]
648084         [female, 16, Fashion, Pisces]
Name: labels, dtype: object

In [60]:
(y_train.append(y_test)).shape


(50000,)

In [61]:
#Dictionary to find count of each label in the multiclass label 

from collections import defaultdict
label_counts = defaultdict(int)
for label in y_train.append(y_test):
    for item in label:
        label_counts[item]+= 1

for key in label_counts:
    print("{} : {}".format(key, label_counts.get(key)))

label_counts.keys()

female : 24619
23 : 5325
indUnk : 18672
Capricorn : 3659
male : 25381
17 : 5986
Student : 11352
Aquarius : 3723
16 : 5437
Sagittarius : 3649
24 : 5850
Aries : 4688
34 : 1552
Cancer : 4726
37 : 665
Leo : 3885
13 : 941
Technology : 3113
Gemini : 3786
33 : 1260
Non-Profit : 1053
Pisces : 3943
35 : 1228
Education : 2125
15 : 3034
14 : 2002
Libra : 4654
26 : 4116
Religion : 371
27 : 3442
Communications-Media : 1423
Arts : 2338
Science : 544
Scorpio : 4176
25 : 4929
Taurus : 4610
45 : 314
Internet : 1162
Virgo : 4501
Government : 504
Marketing : 351
36 : 1019
Fashion : 363
Telecommunications : 281
Law : 641
Engineering : 826
39 : 371
BusinessServices : 341
46 : 214
Consulting : 450
42 : 220
48 : 285
Accounting : 292
RealEstate : 193
Sports-Recreation : 212
Transportation : 176
Agriculture : 84
Publishing : 535
Advertising : 348
Chemicals : 293
43 : 295
40 : 378
38 : 557
Manufacturing : 167
47 : 160
Banking : 297
Museums-Libraries : 229
Automotive : 94
41 : 262
Military : 205
Biotech : 170
44

dict_keys(['female', '23', 'indUnk', 'Capricorn', 'male', '17', 'Student', 'Aquarius', '16', 'Sagittarius', '24', 'Aries', '34', 'Cancer', '37', 'Leo', '13', 'Technology', 'Gemini', '33', 'Non-Profit', 'Pisces', '35', 'Education', '15', '14', 'Libra', '26', 'Religion', '27', 'Communications-Media', 'Arts', 'Science', 'Scorpio', '25', 'Taurus', '45', 'Internet', 'Virgo', 'Government', 'Marketing', '36', 'Fashion', 'Telecommunications', 'Law', 'Engineering', '39', 'BusinessServices', '46', 'Consulting', '42', '48', 'Accounting', 'RealEstate', 'Sports-Recreation', 'Transportation', 'Agriculture', 'Publishing', 'Advertising', 'Chemicals', '43', '40', '38', 'Manufacturing', '47', 'Banking', 'Museums-Libraries', 'Automotive', '41', 'Military', 'Biotech', '44', 'Architecture', 'Tourism', 'HumanResources', 'Construction', 'Environment', 'Maritime', 'InvestmentBanking', 'LawEnforcement-Security'])

### 7. Transform the labels - (7.5 points)

- As we have noticed before, in this task each example can have multiple tags. To deal with
such kind of prediction, we need to transform labels in a binary form and the prediction will be
a mask of 0s and 1s. For this purpose, it is convenient to use MultiLabelBinarizer from sklearn

In [0]:

#### a. Convert your train and test labels using MultiLabelBinarizer

mlb = MultiLabelBinarizer(classes = sorted(label_counts.keys()))

y_train_mlb = mlb.fit_transform(y_train)

y_test_mlb = mlb.fit_transform(y_test)



In [66]:
mlb.classes_


array(['13', '14', '15', '16', '17', '23', '24', '25', '26', '27', '33',
       '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44',
       '45', '46', '47', '48', 'Accounting', 'Advertising', 'Agriculture',
       'Aquarius', 'Architecture', 'Aries', 'Arts', 'Automotive',
       'Banking', 'Biotech', 'BusinessServices', 'Cancer', 'Capricorn',
       'Chemicals', 'Communications-Media', 'Construction', 'Consulting',
       'Education', 'Engineering', 'Environment', 'Fashion', 'Gemini',
       'Government', 'HumanResources', 'Internet', 'InvestmentBanking',
       'Law', 'LawEnforcement-Security', 'Leo', 'Libra', 'Manufacturing',
       'Maritime', 'Marketing', 'Military', 'Museums-Libraries',
       'Non-Profit', 'Pisces', 'Publishing', 'RealEstate', 'Religion',
       'Sagittarius', 'Science', 'Scorpio', 'Sports-Recreation',
       'Student', 'Taurus', 'Technology', 'Telecommunications', 'Tourism',
       'Transportation', 'Virgo', 'female', 'indUnk', 'male'],
      dtype

In [67]:
y_train_mlb[0:2]


array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [68]:
y_test_mlb[0:2]

array([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [0]:
## 8. Choose a classifier - (5 points)

# In this task, we suggest using the One-vs-Rest approach, which is implemented in OneVsRestClassifier class. In this approach k classifiers (= number of tags) are trained. As a
# basic classifier, use LogisticRegression . It is one of the simplest methods, but often it performs good enough in text classification tasks. It might take some time because the
# number of classifiers to train is large.

In [0]:
#### a. Use a linear classifier of your choice, wrap it up in OneVsRestClassifier to train it on every label

logreg_clf = LogisticRegression(solver='lbfgs', max_iter=1000, random_state=42)

In [0]:
#### b. One-vs-Rest approach 

OnevsRest_clf = OneVsRestClassifier(estimator=logreg_clf)

In [72]:
### 9. Fit the classifier, make predictions and get the accuracy (5 points)

#Fit the Classifier on on count vectors for Training dataset

OnevsRest_clf.fit(X_bow_train, y_train_mlb)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=1000,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=42,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [73]:
#classifier's score for training data

OnevsRest_clf.score(X_bow_train, y_train_mlb)

0.21295

In [74]:
#classes present in the multiclass classification

OnevsRest_clf.classes_

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79])

In [75]:
#Predicted values on count vectors for Test dataset

y_pred = OnevsRest_clf.predict(X_bow_test)

y_pred

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

#### a. Print the following
- i. Accuracy score
- ii. F1 score
- iii. Average precision score
- iv. Average recall score
-  v. Tip: Make sure you are familiar with all of them. How would you expect the
things to work for the multi-label scenario? Read about micro/macro/weighted
averaging

- Three metrics are aavilable for evaluation in multiclass classification - macro, micro and weigted.Here, we are using weighted as the parameter

"macro" simply calculates the mean of the binary metrics, giving equal weight to each class. In problems where infrequent classes are nonetheless important, macro-averaging may be a means of highlighting their performance. On the other hand, the assumption that all classes are equally important is often untrue, such that macro-averaging will over-emphasize the typically low performance on an infrequent class.

"weighted" accounts for class imbalance by computing the average of binary metrics in which each class’s score is weighted by its presence in the true data sample.

"micro" gives each sample-class pair an equal contribution to the overall metric (except as a result of sample-weight). Rather than summing the metric per class, this sums the dividends and divisors that make up the per-class metrics to calculate an overall quotient. Micro-averaging may be preferred in multilabel settings, including multiclass classification where a majority class is to be ignored.


In [76]:

#Multilabel Confusion Matrix

print(multilabel_confusion_matrix(y_test_mlb, y_pred))

[[[9766   58]
  [ 167    9]]

 [[9443  162]
  [ 339   56]]

 [[9089  291]
  [ 534   86]]

 [[8446  480]
  [ 878  196]]

 [[8277  509]
  [ 998  216]]

 [[8447  516]
  [ 922  115]]

 [[8322  544]
  [ 987  147]]

 [[8498  477]
  [ 919  106]]

 [[8806  358]
  [ 774   62]]

 [[9003  313]
  [ 614   70]]

 [[9624  103]
  [ 251   22]]

 [[9596  107]
  [ 266   31]]

 [[9673   73]
  [ 225   29]]

 [[9702   74]
  [ 220    4]]

 [[9838   27]
  [ 125   10]]

 [[9869   22]
  [ 105    4]]

 [[9908   21]
  [  70    1]]

 [[9902   20]
  [  58   20]]

 [[9952    7]
  [  39    2]]

 [[9947   15]
  [  35    3]]

 [[9924   24]
  [  49    3]]

 [[9959    6]
  [  33    2]]

 [[9922   19]
  [  55    4]]

 [[9953    5]
  [  39    3]]

 [[9959    9]
  [  26    6]]

 [[9920   15]
  [  59    6]]

 [[9925   15]
  [  56    4]]

 [[9918   13]
  [  68    1]]

 [[9981    6]
  [  13    0]]

 [[8894  356]
  [ 675   75]]

 [[9971    8]
  [  21    0]]

 [[8614  400]
  [ 910   76]]

 [[9357  201]
  [ 402   40]]

 [[9980   

In [0]:
#Classification Report

report = classification_report(y_test_mlb, y_pred, output_dict=True, zero_division=1)

In [78]:
#DataFrame to store Classification Report

cr_df = pd.DataFrame(report).transpose()

cr_df.head(10)

,precision,recall,f1-score,support
0,0.134328,0.051136,0.074074,176.0
1,0.256881,0.141772,0.182708,395.0
2,0.228117,0.138710,0.172518,620.0
3,0.289941,0.182495,0.224000,1074.0
4,0.297931,0.177924,0.222795,1214.0
5,0.182250,0.110897,0.137890,1037.0
6,0.212735,0.129630,0.161096,1134.0
7,0.181818,0.103415,0.131841,1025.0
8,0.147619,0.074163,0.098726,836.0
9,0.182768,0.102339,0.131209,684.0


In [79]:
#Accuracy score

print(accuracy_score(y_test_mlb, y_pred))

0.0161


In [80]:

#F1 score

print(f1_score(y_test_mlb, y_pred, average='weighted'))

0.2794871220803784


In [81]:

#Precision score

print(precision_score(y_test_mlb, y_pred, average='weighted', zero_division=1))

0.3267372578367911


In [82]:
# Recall score

print(recall_score(y_test_mlb, y_pred, average='weighted'))

0.25465


In [83]:
#Average Precision score

print(average_precision_score(y_test_mlb, y_pred, average='weighted'))

0.24581260895544724


In [84]:
#Average Recall score
#support is the number of samples of the true positives that lie in that class.
#avearge of the recall for each calss for the suport for the class weighted by total support for all classes

sum((cr_df.iloc[0:80]['recall']) * (cr_df.iloc[0:80]['support']))/ sum(cr_df.iloc[0:80]['support'])

0.25465

In [85]:
#ROC-AUC score

print(roc_auc_score(y_test_mlb, y_pred, average='weighted'))

0.55384035850559


In [86]:
### 10. Print true label and predicted label for any five examples (7.5 points)

##### True labels for five examples

y_test_mlb



array([[0, 0, 0, ..., 1, 0, 0],
       [0, 1, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 0, 1, 1]])

In [87]:
#Inverse Transform multiclass masked labels to true labels

y_test_labels = mlb.inverse_transform(y_test_mlb)

y_test_labels[0:5]

[('25', 'Arts', 'Virgo', 'female'),
 ('14', 'Sagittarius', 'Student', 'male'),
 ('17', 'Aquarius', 'Consulting', 'female'),
 ('24', 'Pisces', 'indUnk', 'male'),
 ('16', 'Fashion', 'Pisces', 'female')]

In [88]:
y_test[0:5]

276269             [female, 25, Arts, Virgo]
471190      [male, 14, Student, Sagittarius]
526347    [female, 17, Consulting, Aquarius]
129326            [male, 24, indUnk, Pisces]
648084         [female, 16, Fashion, Pisces]
Name: labels, dtype: object

In [89]:
##### Predicted labels for five examples

y_pred

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [90]:
#Predicted Probabilities for test dataset

proba = OnevsRest_clf.predict_proba(X_bow_test)

proba[0]

array([1.82999683e-04, 7.69099186e-07, 3.55843915e-04, 3.48091585e-03,
       5.49494507e-03, 1.45039820e-04, 1.78768703e-02, 1.68893720e-01,
       8.01066441e-03, 1.57072969e-04, 8.64253605e-01, 1.71406022e-06,
       5.92054999e-04, 6.74191498e-02, 8.82206173e-06, 1.04312709e-05,
       1.53901374e-03, 1.63749274e-06, 8.10767708e-02, 7.87973470e-06,
       4.67997622e-06, 9.75646617e-06, 8.92934542e-04, 1.48575860e-04,
       4.77815866e-05, 3.33550504e-07, 4.22964715e-06, 5.44264639e-07,
       1.89728319e-08, 7.32164083e-02, 7.52576093e-08, 3.23757360e-02,
       1.03686476e-01, 1.32384240e-07, 5.46154795e-07, 1.06761354e-05,
       7.58037988e-07, 2.38999896e-03, 5.47390237e-04, 7.26169603e-05,
       4.66796740e-06, 3.53334622e-06, 3.61107110e-05, 1.03806662e-03,
       2.35665145e-03, 1.39624325e-06, 6.84651424e-06, 3.26331396e-04,
       5.01048539e-06, 2.97268675e-06, 5.09418403e-01, 6.90094046e-05,
       5.35474323e-06, 3.65286065e-03, 2.53370262e-02, 1.74661041e-02,
      

In [91]:
#Inverse Transform multiclass probabilities to predicted labels usingig the probabilities above 0.5 threshold

bitarray = OnevsRest_clf.label_binarizer_.inverse_transform(proba, threshold=0.5)
classnames = mlb.inverse_transform(bitarray)

classnames[0:6]

[('33', 'Internet', 'Pisces', 'female'),
 ('male',),
 ('Cancer', 'female'),
 ('female',),
 ('15', 'Student', 'female'),
 ('25', 'Student', 'male')]

In [92]:
#Inverse Transform multiclass masked labels to predicted labels

y_pred_labels = mlb.inverse_transform(y_pred)

y_pred_labels[0:6]

[('33', 'Internet', 'Pisces', 'female'),
 ('male',),
 ('Cancer', 'female'),
 ('female',),
 ('15', 'Student', 'female'),
 ('25', 'Student', 'male')]

#### Observation:
 - Train accuracy is 21.2% and Test accuracy is 16.1%
 - We are not able to get good accuracy when we are applying Bag Of Words model using count vectorizer to extract features
 - When we are combining four different target columns having mutiple classes, the classifier is not able to predict the classes accurately